In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json
from datasets import load_dataset


In [2]:
dataset_orig = load_dataset('relbert/t_rex')

In [38]:
dataset_orig["train"]

Dataset({
    features: ['relation', 'head', 'tail', 'title', 'text'],
    num_rows: 1274264
})

In [3]:
all_relations = set(dataset_orig["train"]["relation"])

In [5]:
all_relations = list(all_relations)

In [6]:
with open('trex_relations2questions2_cleaned.json') as f:
    relations2questions = json.load(f)


In [7]:
all_good_relations = []

for relation in all_relations:
    if relation in relations2questions.keys():
        all_good_relations.append(relation)

In [9]:
def get_points_per_relation(dataset_relations_array, relation):
    return np.where(dataset_relations_array == relation)[0]

dataset_relations_array = np.array(dataset_orig["train"]["relation"])

relations_to_points_dict = {}
for relation in all_good_relations:
    relations_to_points_dict[relation] = get_points_per_relation(dataset_relations_array, relation)

In [19]:

for relation in relations_to_points_dict.keys():
    relations_to_points_dict[relation] = np.array(relations_to_points_dict[relation]).tolist()

In [20]:
#save relations_to_points_dict
with open('trex_relations2idxs.json', 'w') as fp:
    json.dump(relations_to_points_dict, fp)

In [22]:
train_relations = []
train_points = []
ood_relations = []
ood_points = []

for relations in all_good_relations:
    if np.random.rand() < 0.7:
        train_relations.append(relations)
        train_points.append(relations_to_points_dict[relations])
    else:
        ood_relations.append(relations)
        ood_points.append(relations_to_points_dict[relations])

train_points = np.concatenate(train_points)
ood_points = np.concatenate(ood_points)


In [30]:
np.save("custom_trex/train_test_relations.npy", train_relations)
np.save("custom_trex/ood_relations.npy", ood_relations)


In [32]:
np.save("custom_trex/ood_points.npy", ood_points)

In [33]:
train_train_points = []
train_test_points = []

for point in train_points:
    if np.random.rand() < 0.7:
        train_train_points.append(point)
    else:
        train_test_points.append(point)

In [37]:
np.save("custom_trex/train_points.npy", train_train_points)
np.save("custom_trex/test_points.npy", train_test_points)

In [43]:
np.random.shuffle(train_train_points)

In [45]:
np.save("custom_trex/train_points.npy", train_train_points)


In [40]:
small_test_points = np.random.choice(train_test_points, 5000, replace=False)

np.save("custom_trex/test_points_small.npy", small_test_points)

In [41]:
small_ood_points = np.random.choice(ood_points, 5000, replace=False)

np.save("custom_trex/ood_points_small.npy", small_ood_points)